In [ ]:
import numpy as np
import pandas as pd
from make_cache import main, decryption
num_threads = 8
max_n_plus = 6
seed = np.random.randint(10000)
# seed = 2062
# seed = 1986
# seed = 9852
for n_plus in range(1, max_n_plus + 1):
    main(n_plus,num_threads,len_x=40, log_interval=3, seed=seed)

2025-04-10 10:43:13,657 : numba=0.61.0, numpy=2.1.3
2025-04-10 10:43:13,658 : OPT=_OptLevel(3), THREADING_LAYER=default
2025-04-10 10:43:13,660 : USING_SVML=False, ENABLE_AVX=True, DISABLE_JIT=0
2025-04-10 10:43:13,662 : max_op = 1  ,  date : 2025-04-10 10:43:13.662029
2025-04-10 10:43:13,662 : use cores : 8
2025-04-10 10:43:13,663 : making random_x
2025-04-10 10:43:13,663 :    seed = 9955, loop = 100000, upper = 0.5, lower = 1.5
2025-04-10 10:43:13,715 : random_x corrcoef = 3.124852443651029e-06
2025-04-10 10:43:13,716 : make_before_similar_num_list
2025-04-10 10:43:13,717 :    using Memory size =  224.3 MB
2025-04-10 10:43:13,717 :    Memory size of numpy array = 0.0 M bytes +alpha (1data=1280 bytes, loop=2)
2025-04-10 10:43:13,718 :    time : 0:00:00.000621
2025-04-10 10:43:13,718 : make_unique_equations
2025-04-10 10:43:13,718 :    n_op1 = 0, n_op2 = 0
2025-04-10 10:43:13,719 :       make_unique_equations_thread
2025-04-10 10:43:13,719 :          using Memory size =  226.0 MB
2025-

6:4   
51:29   
630:281   
10208:3263   
201551:50154   
[2, 10, 80, 911, 13471, 251705, 5561536]   

In [ ]:
#np.save("arr_len", np.array([2],dtype="int64"))
#np.save("operator_0",np.array([[1],[0]],dtype="int8"))
#np.savez(f"check_change_x_tot_0", np.full((2, 2, 2), -100, dtype="int8"))